<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_Mask2Former_Recopilacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluador Mask2Former para recolección de Métricas
**Recopilación de todas las métricas posibles sin ground truth.**

- Autor: Jesús L.
- Proyecto: TFM. Evaluación comparativa de técnicas de segmentación.

In [27]:
import torch
import numpy as np
import cv2
import json
import time
import psutil
import os
import gc
from datetime import datetime
from pathlib import Path
import hashlib

from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
from skimage import measure
from tqdm import tqdm

In [28]:
# ==========================================
# 1. Montar Google Drive
# ==========================================
from google.colab import drive
drive.mount('/content/drive')

# Ruta a tu dataset en Google Drive
DATASET_PATH = "/content/drive/MyDrive/TFM/mask2former/imagenes"
# Carpeta donde se guardarán los resultados
OUTPUT_PATH = "/content/drive/MyDrive/TFM/mask2former/resultados"

# Crear carpeta de resultados si no existe
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(f"{OUTPUT_PATH}/debug_visualizations", exist_ok=True)

# Lista de modelos a evaluar
MODELOS = [
    "facebook/mask2former-swin-large-coco-instance",
    "facebook/mask2former-swin-base-ade-semantic",
    "facebook/mask2former-swin-small-coco-instance"
]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
class RecolectorMetricasCompletas:
    """
    Recolecta todas las métricas posibles de una segmentación sin ground truth.
    Versión mejorada con debug y detección forzada.
    """
    def __init__(self, config):
        self.config = config
        self.debug_mode = config.get('debug', True)
        self.forzar_deteccion = config.get('forzar_deteccion', True)

        # Usar AutoModelForUniversalSegmentation para todos los modelos mask2former
        try:
            self.processor = AutoImageProcessor.from_pretrained(config['model_name'])
            self.model = AutoModelForUniversalSegmentation.from_pretrained(config['model_name'])
            print(f"Modelo cargado exitosamente: {config['model_name']}")
        except Exception as e:
            print(f"Error cargando modelo {config['model_name']}: {e}")
            raise

        self.device = torch.device(config['device'])
        self.model.to(self.device)
        self.model.eval()

        # Crear directorios
        self.output_dir = Path(config['output_dir'])
        self.output_dir.mkdir(exist_ok=True)
        (self.output_dir / 'datos_completos').mkdir(exist_ok=True)
        (self.output_dir / 'debug_visualizations').mkdir(exist_ok=True)

        self.resultados = []
        print(f"Modelo cargado en {self.device}")
        print(f"Debug mode: {self.debug_mode}")
        print(f"Forzar detección: {self.forzar_deteccion}")

    def preprocesar_imagen(self, ruta, max_size=1024):
        """Preprocesa la imagen redimensionándola si es necesario."""
        img = Image.open(ruta).convert("RGB")
        original_size = img.size
        img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        if self.debug_mode:
            print(f"  Imagen redimensionada de {original_size} a {img.size}")
        return img

    def procesar_imagen(self, ruta_imagen):
        """Procesa una imagen y recolecta TODAS las métricas posibles."""
        inicio = time.time()

        try:
            # 1. CARGA Y ANÁLISIS BÁSICO DE IMAGEN
            imagen_pil = self.preprocesar_imagen(ruta_imagen)
            imagen_np = np.array(imagen_pil)
            h, w = imagen_np.shape[:2]

            # Hash único para la imagen
            ruta_imagen = Path(ruta_imagen)
            hash_img = hashlib.md5(open(ruta_imagen, 'rb').read()).hexdigest()[:12]

            if self.debug_mode:
                print(f"  Procesando imagen: {h}x{w} pixels")

            # Estadísticas básicas de la imagen
            stat = ImageStat.Stat(imagen_pil)
            gray = cv2.cvtColor(imagen_np, cv2.COLOR_RGB2GRAY)

            # 2. ANÁLISIS VISUAL DE LA IMAGEN (sin modelo)
            metricas_imagen = {
                'archivo': os.path.basename(ruta_imagen),
                'hash': hash_img,
                'resolucion_w': w,
                'resolucion_h': h,
                'aspect_ratio': w/h,
                'area_total': w*h,
                'brillo_promedio': float(np.mean(gray)),
                'brillo_std': float(np.std(gray)),
                'rgb_mean_r': float(stat.mean[0]),
                'rgb_mean_g': float(stat.mean[1]),
                'rgb_mean_b': float(stat.mean[2]),
                'rgb_std_r': float(stat.stddev[0]),
                'rgb_std_g': float(stat.stddev[1]),
                'rgb_std_b': float(stat.stddev[2]),
                'varianza_laplacian': float(cv2.Laplacian(gray, cv2.CV_64F).var()),
                'entropia': self._calcular_entropia(gray),
                'densidad_bordes': self._calcular_densidad_bordes(gray),
                'saturacion_media': float(np.mean(cv2.cvtColor(imagen_np, cv2.COLOR_RGB2HSV)[:,:,1])),
                'saturacion_std': float(np.std(cv2.cvtColor(imagen_np, cv2.COLOR_RGB2HSV)[:,:,1])),
                'contraste_local_std': float(np.std([cv2.Laplacian(gray[i:i+32, j:j+32], cv2.CV_64F).var()
                                                   for i in range(0, h-32, 32)
                                                   for j in range(0, w-32, 32) if i+32<h and j+32<w]))
            }

            # 3. RECURSOS ANTES DE INFERENCIA
            recursos_antes = self._obtener_recursos()

            # 4. INFERENCIA DEL MODELO
            inputs = self.processor(images=imagen_pil, return_tensors="pt").to(self.device)

            tiempo_inferencia_inicio = time.time()
            with torch.no_grad():
                outputs = self.model(**inputs)

            tiempo_inferencia = (time.time() - tiempo_inferencia_inicio) * 1000

            if self.debug_mode:
                print(f"  Inferencia completada en {tiempo_inferencia:.1f}ms")

            # 5. POST-PROCESAMIENTO CON MÚLTIPLES UMBRALES Y DEBUG
            umbrales = [0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7]
            resultados_por_umbral = {}

            debug_info = {
                'todas_las_detecciones': {},
                'clases_detectadas': set(),
                'scores_por_clase': {}
            }

            for umbral in umbrales:
                try:
                    # POST-PROCESAMIENTO PRINCIPAL
                    resultado = self.processor.post_process_instance_segmentation(
                        outputs, target_sizes=[(h, w)], threshold=umbral
                    )[0]

                    if self.debug_mode:
                        total_detecciones = len(resultado.get("labels", []))
                        print(f"    Umbral {umbral}: {total_detecciones} detecciones totales")

                        # Debug: mostrar todas las clases detectadas
                        if "labels" in resultado and "scores" in resultado:
                            for label, score in zip(resultado["labels"], resultado["scores"]):
                                clase = int(label.item())
                                score_val = float(score.item())
                                debug_info['clases_detectadas'].add(clase)
                                debug_info['scores_por_clase'].setdefault(clase, []).append(score_val)

                                if self.debug_mode and umbral == 0.1:  # Solo mostrar para un umbral
                                    print(f"      Clase {clase}: score {score_val:.3f}")

                except Exception as e:
                    if self.debug_mode:
                        print(f"    Error en post-processing para umbral {umbral}: {e}")
                    try:
                        # Fallback: segmentación semántica
                        resultado = self.processor.post_process_semantic_segmentation(
                            outputs, target_sizes=[(h, w)]
                        )[0]
                        resultado = self._convertir_semantico_a_instancia(resultado, umbral)
                    except:
                        resultado = {'labels': [], 'masks': [], 'scores': []}

                # Extraer TODAS las detecciones (no solo personas)
                todas_detecciones = []
                mascaras_personas = []
                scores_personas = []

                if 'labels' in resultado and 'masks' in resultado:
                    labels = resultado.get("labels", [])
                    scores = resultado.get("scores", [1.0] * len(labels))
                    masks = resultado.get("masks", [])

                    for i, (label, score) in enumerate(zip(labels, scores)):
                        clase = int(label.item() if hasattr(label, 'item') else label)
                        score_val = float(score.item() if hasattr(score, 'item') else score)

                        todas_detecciones.append({
                            'clase': clase,
                            'score': score_val,
                            'area': int(torch.sum(masks[i]).item()) if i < len(masks) else 0
                        })

                        # Personas (clase 0)
                        if clase == 0:
                            if i < len(masks):
                                mask = masks[i].cpu().numpy().squeeze() > 0.5
                                mascaras_personas.append(mask)
                                scores_personas.append(score_val)

                # FORZAR DETECCIÓN: Si no hay personas pero hay otras detecciones humanas
                if self.forzar_deteccion and len(mascaras_personas) == 0:
                    # Buscar clases que podrían ser personas (0=person en COCO)
                    # Si no hay clase 0, buscar la detección con mayor score que pueda ser humana
                    clases_humanas = [0]  # person
                    detecciones_candidatas = [d for d in todas_detecciones if d['clase'] in clases_humanas]

                    if not detecciones_candidatas and len(todas_detecciones) > 0:
                        # Si no hay detecciones de personas, usar la detección con mayor score
                        mejor_deteccion = max(todas_detecciones, key=lambda x: x['score'])
                        if mejor_deteccion['score'] > 0.01:  # Umbral muy bajo
                            if self.debug_mode:
                                print(f"    FORZANDO: Usando clase {mejor_deteccion['clase']} como persona (score: {mejor_deteccion['score']:.3f})")
                            # Crear máscara ficticia basada en la mejor detección
                            idx_mejor = next(i for i, d in enumerate(todas_detecciones) if d == mejor_deteccion)
                            if idx_mejor < len(masks):
                                mask = masks[idx_mejor].cpu().numpy().squeeze() > 0.5
                                mascaras_personas.append(mask)
                                scores_personas.append(mejor_deteccion['score'])

                # Máscara combinada de todas las personas
                if mascaras_personas:
                    mascara_combinada = np.logical_or.reduce(mascaras_personas)
                else:
                    mascara_combinada = np.zeros((h, w), dtype=bool)

                # Métricas por umbral
                resultados_por_umbral[f'umbral_{umbral}'] = {
                    'num_detecciones': len(mascaras_personas),
                    'num_detecciones_totales': len(todas_detecciones),
                    'todas_las_clases': [d['clase'] for d in todas_detecciones],
                    'todos_los_scores': [d['score'] for d in todas_detecciones],
                    'scores_personas': scores_personas,
                    'area_segmentada': int(np.sum(mascara_combinada)),
                    'porcentaje_imagen': float(np.sum(mascara_combinada) / (h*w) * 100),
                    'coherencia': self._metricas_coherencia(mascara_combinada)
                }

                debug_info['todas_las_detecciones'][f'umbral_{umbral}'] = todas_detecciones

            # 6. CREAR VISUALIZACIÓN DEBUG
            if self.debug_mode:
                self._crear_visualizacion_debug(imagen_np, resultados_por_umbral, hash_img)

            # 7. ANÁLISIS PANÓPTICO
            metricas_panoptico = {}
            try:
                resultado_panoptico = self.processor.post_process_panoptic_segmentation(
                    outputs, target_sizes=[(h, w)]
                )[0]

                if 'segments_info' in resultado_panoptico:
                    segmentos = resultado_panoptico['segments_info']
                    categorias = [seg.get('category_id', -1) for seg in segmentos]

                    metricas_panoptico = {
                        'total_segmentos': len(segmentos),
                        'categorias_unicas': len(set(categorias)),
                        'areas_segmentos': [seg.get('area', 0) for seg in segmentos],
                        'distribucion_categorias': {str(k): categorias.count(k) for k in set(categorias)},
                        'segmentos_persona': sum(1 for seg in segmentos if seg.get('category_id') == 0)
                    }

                    if self.debug_mode:
                        print(f"  Panóptico: {len(segmentos)} segmentos, categorías: {set(categorias)}")

            except Exception as e:
                metricas_panoptico = {'error': f'Panóptico no disponible: {str(e)}'}

            # 8. RECURSOS DESPUÉS DE INFERENCIA
            recursos_despues = self._obtener_recursos()
            tiempo_total = (time.time() - inicio) * 1000

            # 9. ANÁLISIS DE LA MEJOR DETECCIÓN
            mejor_resultado = resultados_por_umbral.get('umbral_0.1', resultados_por_umbral.get('umbral_0.5', {}))

            # 10. CLASIFICACIÓN AUTOMÁTICA DE CONTEXTO
            contexto = self._clasificar_contexto_automatico(
                imagen_np,
                mejor_resultado.get('num_detecciones', 0),
                metricas_imagen
            )

            # 11. RESULTADO FINAL COMPLETO
            resultado_completo = {
                'timestamp': datetime.now().isoformat(),
                'id_procesamiento': f"{hash_img}_{int(time.time())}",
                'imagen': metricas_imagen,
                'rendimiento': {
                    'tiempo_inferencia_ms': tiempo_inferencia,
                    'tiempo_total_ms': tiempo_total,
                    'memoria_antes_mb': recursos_antes.get('memoria_usada_mb', 0),
                    'memoria_despues_mb': recursos_despues.get('memoria_usada_mb', 0),
                    'memoria_gpu_mb': recursos_despues.get('gpu_memoria_mb', 0),
                    'cpu_percent': recursos_despues.get('cpu_percent', 0)
                },
                'segmentacion': resultados_por_umbral,
                'panoptico': metricas_panoptico,
                'contexto': contexto,
                'debug_info': debug_info,
                'modelo': {
                    'nombre': self.config['model_name'],
                    'device': str(self.device),
                    'confianza_umbral_principal': 0.1,
                    'forzar_deteccion': self.forzar_deteccion
                }
            }

            # Debug final
            if self.debug_mode:
                personas_detectadas = mejor_resultado.get('num_detecciones', 0)
                total_detectado = mejor_resultado.get('num_detecciones_totales', 0)
                print(f"  RESULTADO FINAL: {personas_detectadas} personas, {total_detectado} detecciones totales")
                print(f"  Clases detectadas: {sorted(list(debug_info['clases_detectadas']))}")

            del inputs, outputs, imagen_pil, imagen_np
            gc.collect()
            torch.cuda.empty_cache()

            return resultado_completo

        except Exception as e:
            print(f"Error procesando {ruta_imagen}: {e}")
            import traceback
            traceback.print_exc()
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {'archivo': os.path.basename(ruta_imagen)},
                'error': str(e),
                'procesamiento_fallido': True
            }

    def _crear_visualizacion_debug(self, imagen_np, resultados_por_umbral, hash_img):
        """Crea visualizaciones para debug."""
        try:
            fig, axes = plt.subplots(2, 3, figsize=(15, 10))
            axes = axes.flatten()

            # Imagen original
            axes[0].imshow(imagen_np)
            axes[0].set_title('Imagen Original')
            axes[0].axis('off')

            # Resultados por umbral
            umbrales_viz = [0.01, 0.1, 0.3, 0.5, 0.9]
            for i, umbral in enumerate(umbrales_viz):
                if i+1 < len(axes):
                    resultado = resultados_por_umbral.get(f'umbral_{umbral}', {})
                    num_personas = resultado.get('num_detecciones', 0)
                    num_total = resultado.get('num_detecciones_totales', 0)

                    axes[i+1].imshow(imagen_np)
                    axes[i+1].set_title(f'Umbral {umbral}\n{num_personas} personas, {num_total} total')
                    axes[i+1].axis('off')

            plt.tight_layout()
            debug_path = self.output_dir / 'debug_visualizations' / f'debug_{hash_img}.png'
            plt.savefig(debug_path, dpi=150, bbox_inches='tight')
            plt.close()

            print(f"  Debug viz guardada: {debug_path}")

        except Exception as e:
            print(f"  Error creando visualización debug: {e}")

    def _convertir_semantico_a_instancia(self, resultado_semantico, umbral):
        """Convierte resultado de segmentación semántica a formato de instancia."""
        if hasattr(resultado_semantico, 'cpu'):
            mask = resultado_semantico.cpu().numpy()
        else:
            mask = resultado_semantico

        # Encontrar píxeles de personas (asumiendo que la clase 0 es persona)
        persona_mask = (mask == 0)

        if np.any(persona_mask):
            return {
                'labels': [0],
                'masks': [torch.from_numpy(persona_mask.astype(np.float32))],
                'scores': [1.0]
            }
        else:
            return {'labels': [], 'masks': [], 'scores': []}

    def _calcular_entropia(self, imagen_gray):
        """Calcula la entropía de Shannon de la imagen."""
        histogram, _ = np.histogram(imagen_gray, bins=256, range=(0, 256))
        histogram = histogram + 1e-7  # Evitar log(0)
        prob = histogram / np.sum(histogram)
        return float(-np.sum(prob * np.log2(prob)))

    def _calcular_densidad_bordes(self, imagen_gray):
        """Calcula densidad de bordes usando Canny."""
        edges = cv2.Canny(imagen_gray, 50, 150)
        return float(np.sum(edges > 0) / edges.size)

    def _metricas_coherencia(self, mascara):
        """Métricas de coherencia espacial de una máscara."""
        if np.sum(mascara) == 0:
            return {
                'area': 0,
                'componentes': 0,
                'compacidad': 0.0,
                'solidez': 0.0
            }

        # Componentes conectados
        labeled = measure.label(mascara)
        num_componentes = int(labeled.max())

        # Contornos para otras métricas
        contours, _ = cv2.findContours(mascara.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            contorno_principal = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(contorno_principal)
            perimetro = cv2.arcLength(contorno_principal, True)

            # Convex hull
            hull = cv2.convexHull(contorno_principal)
            area_hull = cv2.contourArea(hull)

            # Métricas
            compacidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0
            solidez = area / area_hull if area_hull > 0 else 0

            return {
                'area': int(area),
                'componentes': num_componentes,
                'compacidad': float(compacidad),
                'solidez': float(solidez),
                'perimetro': float(perimetro)
            }

        return {
            'area': int(np.sum(mascara)),
            'componentes': num_componentes,
            'compacidad': 0.0,
            'solidez': 0.0
        }

    def _obtener_recursos(self):
        """Obtiene información de recursos del sistema."""
        try:
            mem = psutil.virtual_memory()
            recursos = {
                'memoria_usada_mb': mem.used / (1024*1024),
                'cpu_percent': psutil.cpu_percent()
            }

            if torch.cuda.is_available():
                recursos['gpu_memoria_mb'] = torch.cuda.memory_allocated() / (1024*1024)

            return recursos
        except:
            return {}

    def _clasificar_contexto_automatico(self, imagen, num_personas, metricas_img):
        """Clasificación automática de contexto basada en métricas."""

        # Reglas simples de clasificación
        if num_personas == 0:
            categoria = 'sin_personas'
        elif num_personas == 1:
            if metricas_img['densidad_bordes'] < 0.1:
                categoria = 'retrato_simple'
            else:
                categoria = 'retrato_complejo'
        else:
            categoria = 'multiples_personas'

        # Análisis de complejidad
        complejidad = (
            metricas_img['densidad_bordes'] * 0.4 +
            min(metricas_img['varianza_laplacian'] / 1000, 1.0) * 0.3 +
            (metricas_img['contraste_local_std'] / 100) * 0.3
        )

        return {
            'categoria': categoria,
            'complejidad_score': float(min(complejidad, 1.0)),
            'num_personas_detectadas': num_personas,
            'iluminacion': 'baja' if metricas_img['brillo_promedio'] < 80 else
                         'alta' if metricas_img['brillo_promedio'] > 180 else 'normal'
        }

    def evaluar_dataset(self, rutas_imagenes):
        """Evalúa un conjunto de imágenes completo."""
        print(f"Iniciando evaluación de {len(rutas_imagenes)} imágenes")

        self.resultados = []
        exitosas = 0

        for i, ruta in enumerate(rutas_imagenes):
            print(f"[{i+1:4d}/{len(rutas_imagenes)}] {os.path.basename(ruta)}")

            resultado = self.procesar_imagen(ruta)
            self.resultados.append(resultado)

            if not resultado.get('procesamiento_fallido', False):
                exitosas += 1
                num_personas = resultado.get('segmentacion', {}).get('umbral_0.1', {}).get('num_detecciones', 0)
                num_total = resultado.get('segmentacion', {}).get('umbral_0.1', {}).get('num_detecciones_totales', 0)
                tiempo = resultado.get('rendimiento', {}).get('tiempo_total_ms', 0)
                print(f"  ✓ {num_personas} personas ({num_total} total), {tiempo:.1f}ms")
            else:
                print(f"  ✗ Error: {resultado.get('error', 'unknown')}")

            # Limpieza periódica de memoria
            if (i + 1) % 10 == 0:
                torch.cuda.empty_cache()

        print(f"\nResumen: {exitosas}/{len(rutas_imagenes)} exitosas")
        return self.resultados

    def guardar_resultados(self, nombre_archivo=None):
        """Guarda todos los resultados en formato JSON."""
        if not nombre_archivo:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_archivo = f"evaluacion_mask2former_debug_{timestamp}.json"

        ruta_archivo = self.output_dir / 'datos_completos' / nombre_archivo

        # Preparar datos para JSON
        datos_exportacion = {
            'resumen': {
                'total_imagenes': len(self.resultados),
                'exitosas': sum(1 for r in self.resultados if not r.get('procesamiento_fallido', False)),
                'timestamp': datetime.now().isoformat(),
                'modelo': self.config['model_name']
            },
            'configuracion': self.config,
            'resultados': self.resultados
        }

        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            json.dump(datos_exportacion, f, indent=2, ensure_ascii=False)

        print(f"Resultados guardados en: {ruta_archivo}")
        print(f"Tamaño del archivo: {os.path.getsize(ruta_archivo) / (1024*1024):.2f} MB")

        return str(ruta_archivo)

In [30]:
# ==========================================
# Funciones auxiliares (sin cambios)
# ==========================================
def cargar_dataset(ruta_dataset, extensiones=(".jpg", ".png", ".jpeg")):
    ruta = Path(ruta_dataset)
    imagenes = [str(p) for p in ruta.glob("**/*") if p.suffix.lower() in extensiones]
    print(f"Dataset cargado: {len(imagenes)} imágenes encontradas.")
    return imagenes

def procesar_por_lotes(evaluador, imagenes, tam_lote=5):
    print("Procesando por lotes...")
    resultados_totales = []
    for i in tqdm(range(0, len(imagenes), tam_lote), desc="Procesando lotes"):
        lote = imagenes[i:i+tam_lote]
        resultados_lote = evaluador.evaluar_dataset(lote)
        resultados_totales.extend(resultados_lote)
        gc.collect()
        torch.cuda.empty_cache()
    return resultados_totales

def generar_resumen(resultados):
    resumen = {}
    for r in resultados:
        for clave, valor in r.items():
            if isinstance(valor, (int, float)):
                resumen.setdefault(clave, []).append(valor)
    return {k: sum(v)/len(v) for k, v in resumen.items() if v}

def ejecutar_multi_modelo_debug(modelos, dataset_path, drive_output_path, tam_lote=1):
    imagenes = cargar_dataset(dataset_path)
    resumen_global = {}

    for modelo in modelos:
        print(f"\n=========================")
        print(f"Ejecutando modelo: {modelo}")
        print(f"=========================")

        try:
            config = {
                'model_name': modelo,
                'device': "cuda" if torch.cuda.is_available() else "cpu",
                'output_dir': drive_output_path,
                'debug': True,
                'forzar_deteccion': True
            }

            evaluador = RecolectorMetricasCompletas(config)
            resultados = procesar_por_lotes(evaluador, imagenes, tam_lote)

            # Guardar resultados por modelo
            nombre_archivo = f"resultados_debug_{modelo.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            ruta_archivo = Path(drive_output_path) / nombre_archivo
            with open(ruta_archivo, 'w', encoding='utf-8') as f:
                json.dump(resultados, f, indent=2, ensure_ascii=False)
            print(f"Resultados guardados en: {ruta_archivo}")

            resumen_global[modelo] = generar_resumen(resultados)

            # Limpieza de memoria entre modelos
            del evaluador
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error con el modelo {modelo}: {e}")
            continue

    # Guardar resumen global
    archivo_resumen = Path(drive_output_path) / f"resumen_global_debug_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(archivo_resumen, 'w', encoding='utf-8') as f:
        json.dump(resumen_global, f, indent=2, ensure_ascii=False)
    print(f"\nResumen global guardado en: {archivo_resumen}")

    # Mostrar resumen de detecciones
    print("\n" + "="*50)
    print("RESUMEN DE DETECCIONES POR MODELO")
    print("="*50)

    for modelo, datos in resumen_global.items():
        print(f"\nModelo: {modelo}")
        print("-" * 40)

        # Buscar métricas de personas detectadas en los resultados originales
        resultados_modelo = []
        for resultado in [r for r in resultados if r.get('modelo', {}).get('nombre') == modelo]:
            if not resultado.get('procesamiento_fallido', False):
                segmentacion = resultado.get('segmentacion', {})
                for umbral, datos_umbral in segmentacion.items():
                    if 'num_detecciones' in datos_umbral:
                        resultados_modelo.append({
                            'umbral': umbral,
                            'personas': datos_umbral.get('num_detecciones', 0),
                            'total': datos_umbral.get('num_detecciones_totales', 0),
                            'clases': datos_umbral.get('todas_las_clases', [])
                        })

        if resultados_modelo:
            # Mostrar mejor resultado por umbral
            for umbral in ['umbral_0.01', 'umbral_0.1', 'umbral_0.5']:
                datos_umbral = next((r for r in resultados_modelo if r['umbral'] == umbral), None)
                if datos_umbral:
                    print(f"  {umbral}: {datos_umbral['personas']} personas, {datos_umbral['total']} total")
                    if datos_umbral['clases']:
                        clases_unicas = list(set(datos_umbral['clases']))
                        print(f"    Clases detectadas: {clases_unicas}")
        else:
            print("  No hay datos de detección disponibles")

def analizar_resultados_detallado(ruta_resultados_json):
    """
    Analiza en detalle los resultados de un archivo JSON para diagnosticar
    por qué no se detectan personas.
    """
    print("\n" + "="*60)
    print("ANÁLISIS DETALLADO DE RESULTADOS")
    print("="*60)

    try:
        with open(ruta_resultados_json, 'r', encoding='utf-8') as f:
            datos = json.load(f)

        if isinstance(datos, list):
            resultados = datos
        else:
            resultados = datos.get('resultados', [])

        for i, resultado in enumerate(resultados):
            if resultado.get('procesamiento_fallido', False):
                continue

            print(f"\nImagen {i+1}: {resultado.get('imagen', {}).get('archivo', 'unknown')}")
            print("-" * 40)

            # Información del modelo
            modelo_info = resultado.get('modelo', {})
            print(f"Modelo: {modelo_info.get('nombre', 'unknown')}")

            # Información de la imagen
            imagen_info = resultado.get('imagen', {})
            print(f"Resolución: {imagen_info.get('resolucion_w', 0)}x{imagen_info.get('resolucion_h', 0)}")
            print(f"Brillo promedio: {imagen_info.get('brillo_promedio', 0):.1f}")

            # Análisis por umbral
            segmentacion = resultado.get('segmentacion', {})
            print("\nDetecciones por umbral:")

            for umbral in ['umbral_0.01', 'umbral_0.1', 'umbral_0.5', 'umbral_0.9']:
                datos_umbral = segmentacion.get(umbral, {})
                personas = datos_umbral.get('num_detecciones', 0)
                total = datos_umbral.get('num_detecciones_totales', 0)
                clases = datos_umbral.get('todas_las_clases', [])
                scores = datos_umbral.get('todos_los_scores', [])

                print(f"  {umbral}: {personas} personas de {total} detecciones totales")

                if clases and scores:
                    # Mostrar las 3 mejores detecciones
                    detecciones_con_score = list(zip(clases, scores))
                    detecciones_ordenadas = sorted(detecciones_con_score, key=lambda x: x[1], reverse=True)

                    print(f"    Top 3 detecciones:")
                    for j, (clase, score) in enumerate(detecciones_ordenadas[:3]):
                        print(f"      {j+1}. Clase {clase}: {score:.4f}")

            # Debug info si está disponible
            debug_info = resultado.get('debug_info', {})
            if debug_info:
                clases_detectadas = debug_info.get('clases_detectadas', set())
                if clases_detectadas:
                    print(f"\nTodas las clases detectadas: {sorted(list(clases_detectadas))}")

                scores_por_clase = debug_info.get('scores_por_clase', {})
                if scores_por_clase:
                    print("Scores promedio por clase:")
                    for clase, scores_lista in scores_por_clase.items():
                        avg_score = sum(scores_lista) / len(scores_lista)
                        max_score = max(scores_lista)
                        print(f"  Clase {clase}: promedio={avg_score:.4f}, máximo={max_score:.4f}")

            # Contexto automático
            contexto = resultado.get('contexto', {})
            if contexto:
                print(f"\nContexto: {contexto.get('categoria', 'unknown')}")
                print(f"Complejidad: {contexto.get('complejidad_score', 0):.3f}")
                print(f"Iluminación: {contexto.get('iluminacion', 'unknown')}")

    except Exception as e:
        print(f"Error analizando resultados: {e}")

def crear_modelo_personalizado_personas():
    """
    Crea un evaluador específico optimizado para detectar personas
    con configuraciones más agresivas.
    """
    print("\n" + "="*50)
    print("CREANDO MODELO OPTIMIZADO PARA PERSONAS")
    print("="*50)

    # Configuración optimizada
    config_optimizada = {
        'model_name': "facebook/mask2former-swin-large-coco-instance",
        'device': "cuda" if torch.cuda.is_available() else "cpu",
        'output_dir': OUTPUT_PATH,
        'debug': True,
        'forzar_deteccion': True,
        'umbrales_personalizados': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
        'optimizado_personas': True
    }

    return RecolectorMetricasCompletas(config_optimizada)

def test_deteccion_simple(ruta_imagen):
    """
    Prueba simple y directa de detección en una imagen específica.
    """
    print(f"\n" + "="*60)
    print(f"TEST DE DETECCIÓN SIMPLE")
    print(f"Imagen: {os.path.basename(ruta_imagen)}")
    print("="*60)

    try:
        # Cargar modelo más simple
        processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-instance")
        model = AutoModelForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-instance")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()

        # Cargar imagen
        image = Image.open(ruta_imagen).convert("RGB")
        print(f"Imagen cargada: {image.size}")

        # Redimensionar si es muy grande
        if max(image.size) > 800:
            image.thumbnail((800, 800), Image.Resampling.LANCZOS)
            print(f"Imagen redimensionada a: {image.size}")

        # Procesar
        inputs = processor(images=image, return_tensors="pt").to(device)
        print("Imagen procesada por el processor")

        with torch.no_grad():
            outputs = model(**inputs)
        print("Inferencia completada")

        # Probar múltiples umbrales
        umbrales = [0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7]

        for umbral in umbrales_test:
            try:
                resultado = processor.post_process_instance_segmentation(
                    outputs, target_sizes=[image.size[::-1]], threshold=umbral
                )[0]

                if "labels" in resultado:
                    labels = resultado["labels"]
                    scores = resultado.get("scores", [])

                    print(f"\nUmbral {umbral}:")
                    print(f"  Total detecciones: {len(labels)}")

                    # Contar personas (clase 0)
                    personas = sum(1 for label in labels if label.item() == 0)
                    print(f"  Personas detectadas: {personas}")

                    # Mostrar todas las detecciones
                    if len(labels) > 0:
                        print("  Todas las detecciones:")
                        detecciones = []
                        for i, (label, score) in enumerate(zip(labels, scores)):
                            clase = label.item()
                            score_val = score.item()
                            detecciones.append((clase, score_val))

                        # Ordenar por score
                        detecciones.sort(key=lambda x: x[1], reverse=True)

                        for i, (clase, score) in enumerate(detecciones[:10]):  # Top 10
                            marca = "👤" if clase == 0 else "🔸"
                            print(f"    {i+1:2d}. {marca} Clase {clase:2d}: {score:.4f}")

                    if personas == 0 and len(labels) > 0:
                        print("  ⚠️  No se detectaron personas, pero sí otros objetos")
                        mejor_deteccion = max([(l.item(), s.item()) for l, s in zip(labels, scores)], key=lambda x: x[1])
                        print(f"  💡 Mejor detección: Clase {mejor_deteccion[0]} con score {mejor_deteccion[1]:.4f}")

                        # ¿Podríamos forzar esta detección como persona?
                        if mejor_deteccion[1] > 0.1:
                            print(f"  🔄 Se podría considerar como persona con forzado")

                else:
                    print(f"Umbral {umbral}: Sin resultados de labels")

            except Exception as e:
                print(f"Error con umbral {umbral}: {e}")

        print(f"\n" + "="*60)
        print("RECOMENDACIONES:")

        # Análisis de la imagen
        img_array = np.array(image)
        brillo_promedio = np.mean(cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY))

        print(f"1. Brillo promedio de la imagen: {brillo_promedio:.1f}")
        if brillo_promedio < 50:
            print("   → Imagen muy oscura, podría afectar la detección")
        elif brillo_promedio > 200:
            print("   → Imagen muy brillante, podría causar sobreexposición")
        else:
            print("   → Brillo adecuado para detección")

        print("2. Para mejorar la detección:")
        print("   → Usar umbrales más bajos (0.01 - 0.1)")
        print("   → Activar forzado de detección")
        print("   → Probar con diferentes modelos")
        print("   → Verificar que la resolución no sea excesiva")

        del model, processor, inputs, outputs
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"Error en test simple: {e}")
        import traceback
        traceback.print_exc()

In [31]:
# ==========================================
# 5. Configuración y ejecución principal
# ==========================================
if __name__ == "__main__":
    print("Ejecutando evaluación con debug mejorado...")

    # Ejecutar evaluación completa
    ejecutar_multi_modelo_debug(MODELOS, DATASET_PATH, OUTPUT_PATH, tam_lote=1)

    # Test adicional en la primera imagen encontrada
    imagenes = cargar_dataset(DATASET_PATH)
    if imagenes:
        print(f"\n{'='*60}")
        print("EJECUTANDO TEST ADICIONAL EN LA PRIMERA IMAGEN")
        print(f"{'='*60}")
        test_deteccion_simple(imagenes[0])

    print(f"\n{'='*60}")
    print("EVALUACIÓN COMPLETADA")
    print(f"{'='*60}")
    print("Archivos generados:")
    print("- Resultados JSON por cada modelo")
    print("- Resumen global")
    print("- Visualizaciones debug en /debug_visualizations/")
    print("- Logs detallados en consola")

    # Instrucciones para análisis posterior
    print(f"\nPara análizar resultados detallados, ejecuta:")
    print(f"analizar_resultados_detallado('/path/to/results.json')")

Ejecutando evaluación con debug mejorado...
Dataset cargado: 1 imágenes encontradas.

Ejecutando modelo: facebook/mask2former-swin-large-coco-instance
Modelo cargado exitosamente: facebook/mask2former-swin-large-coco-instance
Modelo cargado en cuda
Debug mode: True
Forzar detección: True
Procesando por lotes...


Procesando lotes:   0%|          | 0/1 [00:00<?, ?it/s]

Iniciando evaluación de 1 imágenes
[   1/1] _DSC0442.jpg
  Imagen redimensionada de (3849, 5774) a (683, 1024)
  Procesando imagen: 1024x683 pixels
  Inferencia completada en 177.7ms
    Umbral 0.001: 0 detecciones totales
    Umbral 0.01: 0 detecciones totales
    Umbral 0.05: 0 detecciones totales
    Umbral 0.1: 0 detecciones totales
    Umbral 0.3: 0 detecciones totales
    Umbral 0.5: 0 detecciones totales
    Umbral 0.7: 0 detecciones totales


`label_ids_to_fuse` unset. No instance will be fused.


  Debug viz guardada: /content/drive/MyDrive/TFM/mask2former/resultados/debug_visualizations/debug_b11eece7179a.png
  Panóptico: 1 segmentos, categorías: {-1}
  RESULTADO FINAL: 0 personas, 0 detecciones totales
  Clases detectadas: []


...

`label_ids_to_fuse` unset. No instance will be fused.


  Debug viz guardada: /content/drive/MyDrive/TFM/mask2former/resultados/debug_visualizations/debug_b11eece7179a.png
  Panóptico: 1 segmentos, categorías: {-1}
  RESULTADO FINAL: 0 personas, 0 detecciones totales
  Clases detectadas: []
  ✓ 0 personas (0 total), 1960.7ms

Resumen: 1/1 exitosas


Procesando lotes: 100%|██████████| 1/1 [00:02<00:00,  2.80s/it]


Error con el modelo facebook/mask2former-swin-small-coco-instance: Object of type set is not JSON serializable

Resumen global guardado en: /content/drive/MyDrive/TFM/mask2former/resultados/resumen_global_debug_20250901_181213.json

RESUMEN DE DETECCIONES POR MODELO
Dataset cargado: 1 imágenes encontradas.

EJECUTANDO TEST ADICIONAL EN LA PRIMERA IMAGEN

TEST DE DETECCIÓN SIMPLE
Imagen: _DSC0442.jpg
Imagen cargada: (3849, 5774)
Imagen redimensionada a: (533, 800)
Imagen procesada por el processor
Inferencia completada
Error en test simple: name 'umbrales_test' is not defined

EVALUACIÓN COMPLETADA
Archivos generados:
- Resultados JSON por cada modelo
- Resumen global
- Visualizaciones debug en /debug_visualizations/
- Logs detallados en consola

Para análizar resultados detallados, ejecuta:
analizar_resultados_detallado('/path/to/results.json')


Traceback (most recent call last):
  File "/tmp/ipython-input-1266062792.py", line 250, in test_deteccion_simple
    for umbral in umbrales_test:
                  ^^^^^^^^^^^^^
NameError: name 'umbrales_test' is not defined
